In [1]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
# We need to use the connector which is used to connect the local Spark environment to Google storage to access the buckets. 
# Need to create a Google service account and create a role to access GCS buckets specific to a project. The connector .jar file
# was downloaded from https://cloud.google.com/dataproc/docs/concepts/connectors/cloud-storage
credentials_location = './keys/gcs_key.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
# Creare a spark context based on the Google Hadoop file system

sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/03/04 10:38:16 WARN Utils: Your hostname, Hema resolves to a loopback address: 127.0.1.1; using 172.30.190.161 instead (on interface eth0)
24/03/04 10:38:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/04 10:38:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# As before, start a Spark builder session to run jobs

spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

'''
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()
'''

'\nspark = SparkSession.builder     .master("local[*]")     .appName(\'test\')     .getOrCreate()\n'

In [5]:

df_green = spark.read.parquet('data/pq/green/*/*') # Read all the green taxi parquets

In [6]:
df_green.count()

2304517